In [209]:
import pandas as pd
import seaborn as sns
import re
import numpy as np

In [218]:
def get_vector(frame, components, col):

    #number of unique values and dictionary of unique values
    #number = classification.nunique(axis=0)

    components.update(frame[col].value_counts().to_dict())
    vector = np.fromiter(components.values(), dtype = int)
    
    return(vector/vector.sum())

In [219]:
#FASTA metadata upload, faster version of the one in the masterthesis
exclude = [i for i, line in enumerate(open('A.fasta')) if not line.startswith('>')]
metadata = pd.read_csv('A.fasta', skiprows = exclude, sep = '|', na_filter = False, header = None)
metadata.columns = ['accession', 'strain', 'segment', 'protein', 'genus', 'subtype', 'date', 'host', 'curation']

In [220]:
#upload cluster dataframe, join to metadata minus equivalent segment column
cluster = pd.read_csv('cluster.csv', sep = ',', na_filter = False, header = 0, index_col = 0)
intersection = set(metadata.columns) & set(cluster.columns)
classification = cluster.join(metadata.set_index('accession').drop(columns=list(intersection), axis=1))

In [221]:
#Date Column Cleanup and components
classification[['month', 'day', 'year']] = classification['date'].apply(lambda x : x if len(re.findall('/', x)) == 2 else (x.split('/')[0] + '/NA/' + x.split('/')[1] if len(re.findall('/', x)) == 1 else 'NA/NA/' + x)).str.split('/', expand=True)
names = pd.Series({c: classification[c].unique() for c in classification})
components = dict.fromkeys(names['year'], 0)

In [222]:
vectors = classification.groupby(by=['segment', 'cluster']).apply(lambda x: get_vector(x, components, 'year'))

In [226]:
vectors.loc[1]

cluster
-1     [0.0, 0.0, 0.0, 0.06896551724137931, 0.0689655...
 0     [0.0, 0.0, 0.018691588785046728, 0.09345794392...
 1     [4.9348598499802604e-05, 4.9348598499802604e-0...
 2     [5.1007396072430504e-05, 5.1007396072430504e-0...
 3     [5.927330922885425e-05, 5.927330922885425e-05,...
 4     [0.0002508780732563974, 0.0002508780732563974,...
 5     [0.0002516356316054353, 0.0002516356316054353,...
 6     [0.00025144581342720644, 0.0002514458134272064...
 7     [0.0002522704339051463, 0.0002522704339051463,...
 8     [0.0002595380223202699, 0.0002595380223202699,...
 9     [0.0002594706798131811, 0.0002594706798131811,...
 10    [7.597052343690648e-05, 7.597052343690648e-05,...
 11    [9.334453467749463e-05, 9.334453467749463e-05,...
 12    [7.990411506192568e-05, 7.990411506192568e-05,...
 13    [9.3694368968425e-05, 9.3694368968425e-05, 0.0...
 14    [0.00010362694300518135, 0.0001036269430051813...
 15    [0.00010359473738734072, 0.0001035947373873407...
 16    [0.0001054852320